In [ ]:
import pandas as pd
df = pd.read_csv("../Data/df_with_clusters_local_translated_with_clusters.csv")
text = df["claimReviewed"]

In [ ]:
from google.cloud import translate_v3 as translate
from google.oauth2 import service_account
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from tqdm import tqdm

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "XXX"

def translate_text(client, parent, sentence, target_language):
    try:
        response = client.translate_text(
            request={
                "parent": parent,
                "contents": [sentence],
                "mime_type": "text/plain",
                "target_language_code": target_language,
            }
        )
        return sentence, response.translations[0].translated_text
    except Exception as e:
        # You can log the error here or handle it as per your requirements
        print(f"Error translating '{sentence}': {e}")
        return sentence, None

def translate_sentences(sentences, target_language, num_workers=5):
    credentials = service_account.Credentials.from_service_account_file(
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
    )

    client = translate.TranslationServiceClient(credentials=credentials)
    project_id = "XXX"
    parent = f"projects/{project_id}/locations/global"

    # Using a dictionary to map original to translated sentences
    translations = {}

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all translation tasks concurrently
        future_to_sentence = {
            executor.submit(translate_text, client, parent, sentence, target_language): sentence
            for sentence in sentences
        }
        
        # As each translation completes, record it in the translations dictionary
        for future in tqdm(as_completed(future_to_sentence)):
            original = future_to_sentence[future]
            try:
                _, translation = future.result()
                translations[original] = translation
            except Exception as exc:
                print(f"An error occurred with sentence '{original}': {exc}")
                translations[original] = None

    return translations

# Example usage
translated_dict = translate_sentences(
    text,
    'en'
)

df["tranlated_claimReviewed"] = df["claimReviewed"].map(translated_dict)
df.to_csv("../Data/df_with_clusters_local_translated_with_clusters.csv", index=False)